In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import math
from torch.utils.data import TensorDataset, DataLoader

# 시드 고정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [2]:
data = pd.read_csv('/home/aibig25/hong_sj/trb/num.csv')
data = data.fillna(0)

unique_ids = data['sequence_ID'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=41, random_state=42)
train_data = data[data['sequence_ID'].isin(train_ids)]
test_data = data[data['sequence_ID'].isin(test_ids)]

independent_vars = data.columns.difference(['center_x', 'center_y','center_x_ma','center_y_ma', 'ID', 'LC'])
dependent_vars = ['center_y_ma']

scaler = MinMaxScaler()

train_data[independent_vars] = scaler.fit_transform(train_data[independent_vars])
test_data[independent_vars] = scaler.transform(test_data[independent_vars])

X_train = train_data[independent_vars]
y_train = train_data[dependent_vars]

X_test = test_data[independent_vars]
y_test = test_data[dependent_vars]

/tmp/ipykernel_10340/3224682186.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[independent_vars] = scaler.fit_transform(train_data[independent_vars])
/tmp/ipykernel_10340/3224682186.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[independent_vars] = scaler.transform(test_data[independent_vars])


In [3]:
# 입력 및 예측 시퀀스 길이 정의
input_sequence_length = 90
output_sequence_length = 60

def create_sequences(data, input_sequence_length, output_sequence_length):
    X = []
    y = []

    for i in range(len(data) - input_sequence_length - output_sequence_length + 1):
        X.append(data.iloc[i:(i + input_sequence_length)][independent_vars].values)
        y.append(data.iloc[(i + input_sequence_length):(i + input_sequence_length + output_sequence_length)][dependent_vars].values)
    
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, input_sequence_length, output_sequence_length)
X_test, y_test = create_sequences(test_data, input_sequence_length, output_sequence_length)

In [4]:
# 데이터셋을 텐서로 변환
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [5]:
class TrajectoryTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim, lstm_hidden_dim, lstm_layers=1):
        super(TrajectoryTransformer, self).__init__()
        self.model_dim = model_dim
        
        # Linear layer to transform input dimensions to model dimensions
        self.encoder = nn.Linear(input_dim, model_dim)
        self.pos_encoder = PositionalEncoding(model_dim)
        self.tgt_linear = nn.Linear(1, model_dim)  # Linear layer for transforming tgt dimensions
        
        # Define transformer encoder and decoder
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(model_dim, num_heads, lstm_hidden_dim, lstm_layers)
            for _ in range(num_encoder_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayer(model_dim, num_heads, lstm_hidden_dim, lstm_layers)
            for _ in range(num_decoder_layers)
        ])
        
        self.decoder = nn.Linear(model_dim, output_dim)

    def forward(self, src, tgt):
        # Encoding source input
        src = self.encoder(src)
        src = src * math.sqrt(self.model_dim)
        src = self.pos_encoder(src.permute(1, 0, 2))  # Add positional encoding

        # Processing target input
        tgt = tgt.squeeze(-1)
        original_shape = tgt.shape
        tgt = tgt.reshape(-1, 1)
        tgt = self.tgt_linear(tgt)
        tgt = tgt.view(original_shape[0], original_shape[1], -1)
        tgt = tgt * math.sqrt(self.model_dim)
        tgt = self.pos_encoder(tgt.permute(1, 0, 2))  # Add positional encoding

        # Apply encoder and decoder layers
        for layer in self.encoder_layers:
            src = layer(src)
        
        for layer in self.decoder_layers:
            tgt = layer(tgt, src)

        # Decode output and return
        output = self.decoder(tgt.permute(1, 0, 2))
        return output

class TransformerEncoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, lstm_hidden_dim, lstm_layers):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(model_dim, num_heads, dropout=0.1)
        
        # LSTM instead of Feedforward layer
        self.lstm = nn.LSTM(input_size=model_dim, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, batch_first=True, dropout=0.1)
        self.linear = nn.Linear(lstm_hidden_dim, model_dim)
        
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, src):
        # Self-attention
        src2 = self.self_attn(src, src, src)[0]
        src = src + self.dropout(src2)
        src = self.norm1(src)
        
        # LSTM
        lstm_out, _ = self.lstm(src.permute(1, 0, 2))  # LSTM requires (batch_size, seq_length, input_size)
        lstm_out = lstm_out.permute(1, 0, 2)  # Transpose back to (seq_length, batch_size, input_size)
        
        # Linear transformation
        src2 = self.linear(lstm_out)
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src

class TransformerDecoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, lstm_hidden_dim, lstm_layers):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(model_dim, num_heads, dropout=0.1)
        self.multihead_attn = nn.MultiheadAttention(model_dim, num_heads, dropout=0.1)
        
        # LSTM instead of Feedforward layer
        self.lstm = nn.LSTM(input_size=model_dim, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, batch_first=True, dropout=0.1)
        self.linear = nn.Linear(lstm_hidden_dim, model_dim)
        
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.norm3 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, tgt, memory):
        # Self-attention for the target sequence
        tgt2 = self.self_attn(tgt, tgt, tgt)[0]
        tgt = tgt + self.dropout(tgt2)
        tgt = self.norm1(tgt)
        
        # Cross-attention with encoder memory
        tgt2 = self.multihead_attn(tgt, memory, memory)[0]
        tgt = tgt + self.dropout(tgt2)
        tgt = self.norm2(tgt)
        
        # LSTM
        lstm_out, _ = self.lstm(tgt.permute(1, 0, 2))  # LSTM requires (batch_size, seq_length, input_size)
        lstm_out = lstm_out.permute(1, 0, 2)  # Transpose back to (seq_length, batch_size, input_size)
        
        # Linear transformation
        tgt2 = self.linear(lstm_out)
        tgt = tgt + self.dropout(tgt2)
        tgt = self.norm3(tgt)
        return tgt

# 위치 인코딩 추가
class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', self.encoding)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x


In [6]:
# 모델 초기화
input_dim = len(independent_vars)
output_dim = len(dependent_vars)
model_dim = 160
num_heads = 5
num_encoder_layers = 3
num_decoder_layers = 3
lstm_hidden_dim = 50
lstm_layers = 1

model = TrajectoryTransformer(input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim, lstm_hidden_dim, lstm_layers)

# 옵티마이저와 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

/home/aibig25/anaconda3/envs/solar/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
device = torch.device("cuda")
model.to(device)

def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()  # 모델을 훈련 모드로 설정
    for epoch in range(epochs):
        total_loss = 0
        for src, tgt in train_loader:
            src = src.to(device)
            tgt = tgt.to(device)
            optimizer.zero_grad()

            # 모델 출력
            output = model(src, tgt)

            # 손실 계산을 위한 타겟 데이터 조정
            min_length = min(output.size(1), tgt.size(1) - 1)
            adjusted_tgt = tgt[:, 1:min_length+1, :]  # output 길이에 맞게 tgt 조정

            # 손실 계산
            output = output[:, :min_length, :]  # output도 동일한 길이로 조정
            loss = criterion(output, adjusted_tgt)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        average_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {average_loss:.4f}')


def evaluate_model(model, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    total_rmse = 0
    total_mape = 0
    total_count = 0
    
    with torch.no_grad():  # 기울기 계산을 중지하여 메모리 사용량과 계산 속도를 개선
        for src, tgt in test_loader:
            src = src.to(device)
            tgt = tgt.to(device)
            output = model(src, tgt[:, :-1, :])  # 마지막 타임 스텝을 제외하고 입력
            
            # 실제 값과 예측 값을 정렬
            tgt_actual = tgt[:, 1:, :]  # 첫 타임 스텝을 제외한 실제 값
            min_length = min(output.size(1), tgt_actual.size(1))
            output = output[:, :min_length, :]
            tgt_actual = tgt_actual[:, :min_length, :]
            
            # RMSE 계산
            rmse = torch.sqrt(torch.mean((output - tgt_actual) ** 2))
            total_rmse += rmse * output.size(0)  # 배치별 가중치를 더하기
            
            # MAPE 계산
            mape = torch.mean(torch.abs((tgt_actual - output) / tgt_actual)) * 100
            total_mape += mape * output.size(0)  # 배치별 가중치를 더하기
            
            total_count += output.size(0)
    
    average_rmse = total_rmse / total_count
    average_mape = total_mape / total_count
    print(f'Final Test RMSE: {average_rmse:.4f}')
    print(f'Final Test MAPE: {average_mape:.4f}')

In [8]:
train_model(model, train_loader, optimizer, criterion, epochs=50)

Epoch 1/50, Loss: 23429.7939
Epoch 2/50, Loss: 1241.6764
Epoch 3/50, Loss: 1227.1008
Epoch 4/50, Loss: 1226.0336
Epoch 5/50, Loss: 1225.5636
Epoch 6/50, Loss: 1225.0773
Epoch 7/50, Loss: 1226.8055
Epoch 8/50, Loss: 1226.0289
Epoch 9/50, Loss: 1226.6106
Epoch 10/50, Loss: 1225.7712
Epoch 11/50, Loss: 1225.8550
Epoch 12/50, Loss: 1225.8640
Epoch 13/50, Loss: 1225.9825
Epoch 14/50, Loss: 1226.3493
Epoch 15/50, Loss: 1227.0028
Epoch 16/50, Loss: 1225.6381
Epoch 17/50, Loss: 1225.5249
Epoch 18/50, Loss: 1225.6852
Epoch 19/50, Loss: 1224.8153
Epoch 20/50, Loss: 1225.3346
Epoch 21/50, Loss: 1225.8363
Epoch 22/50, Loss: 1227.0275
Epoch 23/50, Loss: 1226.0296
Epoch 24/50, Loss: 1226.8763
Epoch 25/50, Loss: 1225.3006
Epoch 26/50, Loss: 1225.3433
Epoch 27/50, Loss: 1226.0391
Epoch 28/50, Loss: 1225.5068
Epoch 29/50, Loss: 1225.3240
Epoch 30/50, Loss: 1225.1863
Epoch 31/50, Loss: 1225.2325
Epoch 32/50, Loss: 1225.3953
Epoch 33/50, Loss: 1224.9454
Epoch 34/50, Loss: 1225.1038
Epoch 35/50, Loss: 122

In [9]:
evaluate_model(model, test_loader)

Final Test RMSE: 27.6250
Final Test MAPE: 5.4223
